<a href="https://colab.research.google.com/github/therealfortune/tensorflow-fortune/blob/master/ImdbReviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np

In [4]:
import tensorflow_datasets as tfds
imdb, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

In [6]:
train_data, test_data = imdb['train'], imdb['test']

In [10]:
print(train_data)
print()
for obj in train_data.take(2):
  print(obj)

<PrefetchDataset shapes: ((), ()), types: (tf.string, tf.int64)>

(<tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'I have been known to fall asleep during films, but this is usually due to a combinatio

In [25]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())

for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [26]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 10000
max_length = 120
oov_tok = "<OOV>" 
embedding_dim = 16
trunc_type = "post"

tokenizer = Tokenizer(num_words=vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen = max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
test_padded = pad_sequences(testing_sequences, maxlen = max_length)

In [27]:
reverse_word_index = dict([(value,key) for (key, value) in word_index.items()])

def decode_review(text):
  return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[4]))
print(training_sentences[4])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? as others have mentioned all the women that go nude in this film are mostly absolutely gorgeous the plot very <OOV> shows the <OOV> of the female <OOV> when men are around they want to be pursued but when no men are around they become the <OOV> of a 14 year old boy and the boy becomes a man really fast we should all be so lucky at this age he then gets up the courage to pursue his true love
As others have mentioned, all the women that go nude in this film are mostly absolutely gorgeous. The plot very ably shows the hypocrisy of the female libido. When men are around they want to be pursued, but when no "men" are around, they become the pursuers of a 14 year old boy. And the boy becomes a man really fast (we should all be so lucky at this age!). He then gets up the courage to pursue his true love.


In [28]:
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(6, activation="relu"),
  tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss= 'binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 120, 16)           160000    
                                                                 
 flatten_1 (Flatten)         (None, 1920)              0         
                                                                 
 dense_2 (Dense)             (None, 6)                 11526     
                                                                 
 dense_3 (Dense)             (None, 1)                 7         
                                                                 
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [29]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(test_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 8s 8ms/step - loss: 0.4919 - accuracy: 0.7441 - val_loss: 0.3430 - val_accuracy: 0.8498
Epoch 2/10
782/782 [==============================] - 6s 7ms/step - loss: 0.2436 - accuracy: 0.9066 - val_loss: 0.3684 - val_accuracy: 0.8406
Epoch 3/10
782/782 [==============================] - 6s 7ms/step - loss: 0.1047 - accuracy: 0.9711 - val_loss: 0.4409 - val_accuracy: 0.8285
Epoch 4/10
782/782 [==============================] - 6s 7ms/step - loss: 0.0306 - accuracy: 0.9954 - val_loss: 0.5149 - val_accuracy: 0.8298
Epoch 5/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0092 - accuracy: 0.9990 - val_loss: 0.5737 - val_accuracy: 0.8296
Epoch 6/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0038 - accuracy: 0.9996 - val_loss: 0.6371 - val_accuracy: 0.8266
Epoch 7/10
782/782 [==============================] - 6s 7ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.6747 - val_accuracy: 0.8301
Epoch 

In [31]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [32]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [33]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>